# Modelling - Part 1

In [ ]:
#import libraries
import pandas as pd
import numpy as np
import collections
import re
import sys
import time
import matplotlib.pyplot as plt
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
import wordninja
from itertools import islice
import pickle
from sklearn.model_selection import train_test_split

In [ ]:
df=pd.read_pickle("df_add_feat") # reload df from pickle

## 1. TFIDF-Vectorization

In [ ]:
#apply tfidf vectorizer after count vecotizer with n_gram_range==1

In [ ]:
#count vectorizer
from sklearn.feature_extraction.text import CountVectorizer

def dummy(tokens):
    return tokens

cv = CountVectorizer(
        tokenizer=dummy,
        preprocessor=dummy,
        ngram_range=(1, 1)
    )  
tokens=df.tokens
x = cv.fit_transform(tokens)
words = cv.get_feature_names()
len(words)

In [ ]:
#tfidf vectorizer 
vectorizer = TfidfVectorizer(tokenizer=dummy,
        preprocessor=dummy,
        ngram_range=(1, 1))

X_tfidf = vectorizer.fit(tokens.values) 

idf_scores = X_tfidf.idf_

In [ ]:
#make histogram of idf scores
maxi=np.max(idf_scores)
mean=np.mean(idf_scores)
min=np.min(idf_scores)
print(maxi, mean, min)
len(idf_scores)

In [ ]:
import seaborn as sns
sns.histplot(data=idf_scores)

In [ ]:
X_tfidf.get_feature_names()[:10]

In [ ]:
#filter out ngrams that occur in too few or too many songs

filtered_indices = np.argwhere(((idf_scores>6) ))
filtered_indices = [idx[0] for idx in filtered_indices]

#list of vocabulary from the vectorizer
vocabulary = X_tfidf.get_feature_names()

#preparing a list with filtered vocabulary
filtered_voc = [vocabulary[i] for i in filtered_indices]

#size before and after filtering
print("no of words before filtering: ", len(idf_scores))
print("no of words after filtering: ", len(filtered_indices))

In [ ]:
filtered_voc[:10]

In [ ]:
#keep only filtered indices
X_tfidf=vectorizer.fit_transform(tokens.values)[:,filtered_indices]
type(X_tfidf)
print(X_tfidf.shape)

## 2. Tf-idf feature selection using tuned random forests

In [ ]:
df=pd.read_pickle("df_add_feat") # reload df from pickle

In [ ]:
#random forest model
#define parameter grid for randomized search with forest model
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

#n_estimators
n_estimators = [250,300, 350]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]


# Create the random grid
random_grid = {'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
               'n_estimators':n_estimators}


In [ ]:
def rf_rs(X_train, X_test, y_train, y_test):

    #Instantiate the classifier
    rf=RandomForestClassifier(n_jobs=-1)
    rs=RandomizedSearchCV(rf,random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1, scoring='roc_auc')
    rs.fit(X_train, y_train)
    return rs

In [ ]:
# fit tuned model
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
def fit_tuned_rf_model(model):
    print("tuned hyperparameters :(best parameters) ", model.best_params_)
    print("ROC AUC :", model.best_score_)

    modelCV=RandomForestClassifier(max_features=model.best_params_['max_features'], max_depth=model.best_params_['max_depth'], min_samples_split=model.best_params_['min_samples_split'], min_samples_leaf=model.best_params_['min_samples_leaf'], bootstrap=model.best_params_['bootstrap'],n_estimators=model.best_params_['n_estimators'])
    modelCV.fit(X_train, y_train)

    #return predicted probabilities
    probs = modelCV.predict_proba(X_test)
        # keep probabilities for the positive outcome only
    probs = probs[:, 1]
    roc_auc=roc_auc_score(y_test, probs)
    fpr, tpr, _ = roc_curve(y_test, probs)
    print("Fitted tuned random forest model ", roc_auc)
    return modelCV, roc_auc, fpr, tpr

In [ ]:
#save labels: ranking<50=1, 0 otherwise
y=pd.cut(df.Rank,bins=[0,50,100],labels=[1,0])
df['y']=y
df[['y','Rank']].groupby('y').mean()
len(y)
df.Rank.groupby(y).mean()

In [ ]:
#Random forest classifier on tfidf features
X_train, X_test, y_train, y_test = train_test_split(X_tfidf,y, test_size=0.33, random_state=42)
rf=rf_rs(X_train, X_test, y_train, y_test)

In [ ]:
# save the tuned model to disk
filename = 'tuned_rf_model.sav'
pickle.dump(rf, open(filename, 'wb'))

In [ ]:
#fit tuned model
rf_fit=fit_tuned_rf_model(rf)

In [ ]:
#plot most important features
importance=rf_fit[0].feature_importances_
feat_dict=dict(zip(filtered_voc,importance))
sort={k: v for k, v in sorted(feat_dict.items(), reverse=True, key=lambda item: item[1])}
n_items=list(islice(sort.items(), 50))

In [ ]:
coeffs=[round(i[1],5) for i in n_items]
indices=[i[0] for i in n_items]


In [ ]:
plt.figure(figsize=(12,12))
plt.barh(indices, coeffs)

In [ ]:
#Select top 1000 features
top_1000=list(sort.items())[:1000]
top_1000_feat=[i[0] for i in top_1000]
#select top feat from X_tfidf
#get indices of top1000 features from filtered_voc
top_1000_indices = [i for i, value in enumerate(filtered_voc) if value in top_1000_feat]
X_1000=X_tfidf[:,top_1000_indices]
X_1000.shape

## 5. Join music features, lyrics features and tfidf features

In [ ]:
music_features=['danceability', 'key', 'loudness','energy','acousticness', 'speechiness', 'mode', 'instrumentalness', 'liveness','valence', 'tempo']
X_music=df[music_features]
X_music=X_music.fillna(X_music.mean())


In [ ]:
#list of lyrics features

add_features=['word_length', 'unique_words', 'total_words','count_rhyming_words']

In [ ]:
#join lyrics data with musical features
#stack sparse matrices
from scipy.sparse import hstack
import scipy.sparse

X_m=scipy.sparse.csr_matrix(X_music.values)
print(X_m.shape)

X_l=scipy.sparse.csr_matrix(df[add_features].values)


In [ ]:
print(X_1000.shape)

In [ ]:
#array of column names
columns=filtered_voc+music_features+add_features
col_names = np.array(columns)
col_names

In [ ]:
X_text=hstack([X_1000, X_l])
X_all=hstack([X_text,X_m])

## 5. Build models

### Hyperparameter tuning

#### Logistic regression

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
from sklearn.linear_model import LogisticRegression

In [ ]:
#define parameter grid for logistic cv

C = [0.00001,0.0001, 0.001, 0.01, 0.1,1,10,100,1000, 10000]
penalty = ['l2']

parameters = dict(C=C, penalty=penalty)

In [ ]:
def logistic_cv(X_train, X_test, y_train, y_test):
    logistic=LogisticRegression(max_iter=500)
    gsl=GridSearchCV(logistic, parameters, cv=3, n_jobs=-1, scoring="roc_auc")
    gsl.fit(X_train, y_train)
    return gsl

In [ ]:
#fit tuned model
def fit_tuned_lr_model(model):    
    logisticCV=LogisticRegression(C=model.best_params_['C'], penalty=model.best_params_['penalty'], max_iter=500)
    logisticCV.fit(X_train, y_train)

    #return predicted probabilities
    lr_probs = logisticCV.predict_proba(X_test)

    # keep probabilities for the positive outcome only
    lr_probs = lr_probs[:, 1]
    roc_auc=roc_auc_score(y_test, lr_probs)
    fpr, tpr, _ = roc_curve(y_test, lr_probs)
    print("tuned hyperparameters :(best parameters) ", model.best_params_)
    print("ROC AUC :", model.best_score_)
    print("Fitted tuned logit  model ", roc_auc)
    return logisticCV,roc_auc, fpr, tpr

In [ ]:
#Logistic Classifier on tfidf features 
X_train, X_test, y_train, y_test = train_test_split(X_tfidf,y, test_size=0.33, random_state=42)
lr=logistic_cv(X_train, X_test, y_train, y_test)
#fit tuned model
lr_fit=fit_tuned_lr_model(lr)
# save the tuned model to disk
filename = 'tuned_logistic_tfidf.sav'
pickle.dump(lr, open(filename, 'wb'))

In [ ]:
#Logistic Classifier on tfidf top 1000 features only 
X_train, X_test, y_train, y_test = train_test_split(X_1000,y, test_size=0.33, random_state=42)
lr=logistic_cv(X_train, X_test, y_train, y_test)

In [ ]:
#fit tuned model
lr_fit_1000=fit_tuned_lr_model(lr)

In [ ]:
# save the tuned model to disk
filename = 'tuned_logistic_1000.sav'
pickle.dump(lr, open(filename, 'wb'))

In [ ]:
#Logistic Classifier on tfidf top 1000 plus engineered text features  
X_train, X_test, y_train, y_test = train_test_split(X_text,y, test_size=0.33, random_state=42)
lr=logistic_cv(X_train, X_test, y_train, y_test)

In [ ]:
# save the tuned model to disk
filename = 'tuned_logistic_text.sav'
pickle.dump(lr, open(filename, 'wb'))

In [ ]:
#fit tuned model
lr_fit_text=fit_tuned_lr_model(lr)

In [ ]:
#Logistic Classifier on tfidf top 1000 plus engineered text features plus music features
X_train, X_test, y_train, y_test = train_test_split(X_all,y, test_size=0.33, random_state=42)
lr=logistic_cv(X_train, X_test, y_train, y_test)

In [ ]:
#fit tuned model
lr_fit_all=fit_tuned_lr_model(lr)

#### Random forest

In [ ]:
#Random forest classifier on tfidf and text features
X_train, X_test, y_train, y_test = train_test_split(X_text,y, test_size=0.33, random_state=42)
rf=rf_rs(X_train, X_test, y_train, y_test)

In [ ]:
# save the tuned model to disk
filename = 'tuned_rf_only_text.sav'
pickle.dump(rf, open(filename, 'wb'))

In [ ]:
#fit tuned model
rf_fit_text=fit_tuned_rf_model(rf)

In [ ]:
#Random forest classifier on all 1000 tfidf plus text and musical features
X_train, X_test, y_train, y_test = train_test_split(X_all,y, test_size=0.33, random_state=42)
rf=rf_rs(X_train, X_test, y_train, y_test)

In [ ]:
# save the tuned model to disk
filename = 'tuned_rf_all.sav'
pickle.dump(rf, open(filename, 'wb'))

In [ ]:
#fit tuned model
rf_fit_all=fit_tuned_rf_model(rf)

### SVM

In [ ]:
#support vector mechines (SVM)
from sklearn.svm import SVC
#initiate model
svm = SVC(probability=True)

In [ ]:
#tune hyperparameters with GridSearchCV
# defining parameter range
X_train, X_test, y_train, y_test = train_test_split(X_all,y, test_size=0.33, random_state=42)

param_grid = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']} 
  
svm_cv = GridSearchCV(svm, param_grid, cv=3, n_jobs=-1, scoring="roc_auc")
  
# fitting the model for grid search
svm_cv.fit(X_train, y_train)

print("Tuned SVM Params: {}".format(svm_cv.best_params_)) 
print("Best score is {}".format(svm_cv.best_score_))

In [ ]:
#make model with tuned parameters
svm_tuned=SVC(C=svm_cv.best_params_['C'], gamma= svm_cv.best_params_['gamma'], kernel= svm_cv.best_params_['kernel'], probability=True)

In [ ]:
svm_tuned.fit(X_train, y_train)

In [ ]:
svm_probs = svm_tuned.predict_proba(X_test)

# keep probabilities for the positive outcome only
svm_probs = svm_probs[:, 1]
roc_auc=roc_auc_score(y_test, svm_probs)
fpr, tpr, _ = roc_curve(y_test, svm_probs)

print("Fitted tuned svm model ", roc_auc)

## Evaluation

In [ ]:
#drawing roc curve and auc


from matplotlib import pyplot

# generate a no skill prediction (majority class)
ns_probs = [0 for _ in range(len(y_test))]

## plot the roc curve: start with no skill prediction
fig, ax = plt.subplots(figsize=(15, 10))
ns_auc = roc_auc_score(y_test, ns_probs) #no skill
ns_fpr, ns_tpr, _ = roc_curve(y_test, ns_probs)
plt.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')


#draw roc curves for tuned models
def draw_roc_curve(model, label):
    model_name=type(model[0]).__name__
    # retrieve roc auc scores
    auc = model[1]
    # calculate roc curves
    # get tpr, fpr
    fpr=model[2]
    tpr=model[3]
    print('Model=%s; Data=%s; ROC AUC=%.3f' % (model_name, label, auc)) 
    lab=model_name+" "+label
    plt.plot(fpr, tpr, marker='.', label=lab)

    
model_dict={'all tfidf features':lr_fit_tfidf, 'top 1000 tfidf features': lr_fit_1000, 'top tfidf plus text features': lr_fit_text,'all features': lr_fit_all, 'top 1000 tfidf features': rf_fit, 'top tfidf plus text features': rf_fit_text,'all features': rf_fit_all}


#WHY CAN I NOT DRAW ALL LINES IN ONE GRAPH USING A LOOP?
for k,v in model_dict.items():
    draw_roc_curve(v,k)

# axis labels
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
# show the legend
plt.legend()
# show the plot
plt.show()
